In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Reshape, Input, Flatten
from sklearn.model_selection import train_test_split

In [2]:
import time, datetime
from fyers_apiv3 import fyersModel
client_id = "ZHQ4IJL7TI-100"
with open("access_token", "r") as f:
    access_token = f.read()

fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

In [110]:
import matplotlib.pyplot as plt
import numpy as np
import os

In [4]:
from indicators.syncind import SyncInd
from indicators.classic import SMA, Alligator, SmoothMA, RSI, MACD, ATR, WMA, EMA, RMA, VolumeROC, KAMA
from indicators.candle import OHLC

In [5]:
date_today = datetime.datetime.now().strftime("%Y-%m-%d")
date_yesterday = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
date_100_p = (datetime.datetime.now() - datetime.timedelta(days=100)).strftime("%Y-%m-%d")
date_yesterday, date_100_p

('2024-11-25', '2024-08-18')

In [6]:
response = fyers.history(data={
                                "symbol": "NSE:ZEEL-EQ",
                                "resolution": "1",
                                "date_format": "1",
                                "range_from": date_100_p,
                                "range_to": date_yesterday,
                                "cont_flag": "1"
                                })

len(response['candles'])

25185

In [7]:
date_temp = []
final_candles_list = []
for i in response['candles']:
    if i[0] not in date_temp:
        final_candles_list.append(i)
        date_temp.append(i[0])
del date_temp
len(final_candles_list)

25185

In [8]:
candles = np.array(final_candles_list)
candles.shape

(25185, 6)

In [9]:
sync = SyncInd(
    SMA(5),
    # Alligator(show_jaw=False, show_teeth=False)
    KAMA(highlight=True)
)

for c in candles:
    sync.append(c)
sync.data().shape

(25185, 9)

In [10]:
model = tf.keras.models.load_model("26NOV_2.keras")
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 8, 200)         │       166,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 8, 200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 150)            │       210,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 150)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           755 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 5)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,133,267 (4.32 MB)

 Trainable params: 377,755 (1.44 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 755,512 (2.88 MB)

In [130]:
response_rt = fyers.history(data={
                                "symbol": "NSE:ZEEL-EQ",
                                "resolution": "1",
                                "date_format": "1",
                                "range_from": date_today,
                                "range_to": date_today,
                                "cont_flag": "1"
                                })

# the following comment code checks for the duplicates in the api data
# uncomment it if duplicates in api response
'''
date_temp = []
final_candles_list_rt = []
for i in response_rt['candles']:
    if i[0] not in date_temp:
        final_candles_list_rt.append(i)
        date_temp.append(i[0])
    else:
        print("duplicate detected ")
del date_temp
'''       
candles = np.array(response_rt['candles'])
for i in candles:
    if sync.sync(i):
        print(datetime.datetime.fromtimestamp(int(i[0])).strftime("%D %H:%M:%S"))

375

In [86]:
for s in sync.data()[-10:, 0]:
    print(datetime.datetime.fromtimestamp(int(s)).strftime("%D %H:%M:%S"))

11/26/24 14:21:12
11/26/24 14:21:12
11/26/24 14:23:20
11/26/24 14:23:20
11/26/24 14:23:20
11/26/24 14:23:20
11/26/24 14:23:20
11/26/24 14:23:20
11/26/24 14:23:20
11/26/24 14:25:28


In [17]:
for i in candles:
    if sync.sync(i):
        print(datetime.datetime.fromtimestamp(int(i[0])).strftime("%D %H:%M:%S"))

In [64]:
datetime.datetime.fromtimestamp(int(sync.last_candle()[0])).strftime("%D %H:%M") == datetime.datetime.fromtimestamp(int(candles[-1][0])).strftime("%D %H:%M")

True

In [109]:
for i in np.array(response['candles'])[-10:, 0]:
    print(datetime.datetime.fromtimestamp(int(i)).strftime("%D %H:%M:%S"))

11/25/24 15:20:00
11/25/24 15:21:00
11/25/24 15:22:00
11/25/24 15:23:00
11/25/24 15:24:00
11/25/24 15:25:00
11/25/24 15:26:00
11/25/24 15:27:00
11/25/24 15:28:00
11/25/24 15:29:00


In [132]:
len(candles)


375